# Part 2: Online aDBS experiment data analysis

The following notebook analyzes 12 days of data collected using movement-responsive adaptive stimulation.

This includes all statistical analyses and figures generated for the second half (experimental half) of the paper Dixon et al., 2025: "Movement-responsive deep brain stimulation for Parkinson’s Disease using a remotely optimized neural decoder." Figures 5-7 and the accompanying analyses are represented, which are labeled in the markdown.

In [ ]:
import pandas as pd
import numpy as np
from itertools import product

from ipywidgets import *
from IPython.display import clear_output, display
from tkinter import Tk, filedialog

from pingouin import ancova, anova
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.contrast import ContrastResults
from scipy.stats import ttest_ind, pearsonr, binomtest

from rcssim import rcs_sim as rcs
from move_adbs_utils import *
from plotting_utils import *
from utils import *

import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib widget

## Plot example session

### Import curated experimental data

In [ ]:
# Select a folder containing the curated training data
choose_folder_button = widgets.Button(description='Choose neural data folder')
display(choose_folder_button)

output = widgets.Output()

@output.capture(clear_output=False, wait=True)
def choose_folder(b):
    clear_output()
    root = Tk()
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)
    b.folder = filedialog.askdirectory()
    global my_folder
    my_folder = b.folder
    print(b.folder)

choose_folder_button.on_click(choose_folder)
display(output)

t = type('test', (object,), {})()
choose_folder(t)

In [ ]:
# Load the neural data
rcs_left = pd.read_csv(my_folder + '/rcs_left.csv')
rcs_right = pd.read_csv(my_folder + '/rcs_right.csv')

### Load the Apple Watch and pose data
watch_left = pd.read_csv(my_folder + '/watch_left.csv')
watch_right = pd.read_csv(my_folder + '/watch_right.csv')

# Load the self scores
self_score = pd.read_csv(my_folder + '/self_score.csv')

### Figure 5b: Online movement-responsive aDBS example

In [ ]:
%matplotlib widget

session = 4

# Plot example segment
plt.style.use('default')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
fig, ax = plt.subplots(4,2, figsize=(6.5,4.5), sharex=True)

t0 = rcs_left.loc[rcs_left.session_id==session].timestamp.values[0]

ax[0,0].set_title('Left Hemisphere')
ax[0,0].plot(watch_right.loc[watch_right.session_id==session].timestamp.values-t0, 
             np.clip(np.log10(watch_right.loc[watch_right.session_id==session].accel.values), -6, 0))
ax[0,0].set_ylabel('True \n acceleration \n (Log-power)')
ax[1,0].plot(rcs_left.loc[rcs_left.session_id==session].timestamp.values-t0, 
             rcs_left.loc[rcs_left.session_id==session].output.values, 
             color='tab:orange')
ax[1,0].set_ylabel('Predicted \n acceleration \n (Z-score)')

watch_idx = (watch_right.timestamp.values-t0 > 965) & (watch_right.timestamp.values-t0 < 1165)
rcs_idx = (rcs_left.timestamp.values-t0 > 965) & (rcs_left.timestamp.values-t0 < 1165)
ax[2,0].fill_between(watch_right.loc[(watch_right.session_id==session) & watch_idx].timestamp.values-t0, 
                     (np.log10(watch_right.loc[(watch_right.session_id==session) & watch_idx].accel.values)>-3) + 0.02, 
                     alpha=0.5, step='mid', label='True', clip_on=False, zorder=10)
ax[2,0].fill_between(rcs_left.loc[(rcs_left.session_id==session) & rcs_idx].timestamp.values-t0, 
                     rcs_left.loc[(rcs_left.session_id==session) & rcs_idx].state.values + 0.02, 
                     alpha=0.5, step='mid', label='Predicted', clip_on=False, zorder=10)
ax[2,0].set_ylabel('Movement \n state')
ax[3,0].plot(rcs_left.loc[rcs_left.session_id==session].timestamp.values-t0, 
             rcs_left.loc[rcs_left.session_id==session].stim.values, 
             color='tab:orange')
ax[3,0].set_ylabel('Stimulation \n (mA)')

ax[0,1].set_title('Right Hemisphere')
ax[0,1].plot(watch_left.loc[watch_left.session_id==session].timestamp.values-t0, 
             np.clip(np.log10(watch_left.loc[watch_left.session_id==session].accel.values), -6, 0))
ax[1,1].plot(rcs_right.loc[rcs_right.session_id==session].timestamp.values-t0, 
             rcs_right.loc[rcs_right.session_id==session].output.values, 
             color='tab:orange')

watch_idx = (watch_left.timestamp.values-t0 > 965) & (watch_left.timestamp.values-t0 < 1165)
rcs_idx = (rcs_right.timestamp.values-t0 > 965) & (rcs_right.timestamp.values-t0 < 1165)
ax[2,1].fill_between(watch_left.loc[(watch_left.session_id==session) & watch_idx].timestamp.values-t0, 
                     (np.log10(watch_left.loc[(watch_left.session_id==session) & watch_idx].accel.values)>-3) + 0.02, 
                     alpha=0.5, step='mid', label='True', clip_on=False, zorder=10)
ax[2,1].fill_between(rcs_right.loc[(rcs_right.session_id==session) & rcs_idx].timestamp.values-t0, 
                     rcs_right.loc[(rcs_right.session_id==session) & rcs_idx].state.values + 0.02, 
                     alpha=0.5, step='mid', label='Predicted', clip_on=False, zorder=10)
ax[3,1].plot(rcs_right.loc[rcs_right.session_id==session].timestamp.values-t0, 
             rcs_right.loc[rcs_right.session_id==session].stim.values, 
             color='tab:orange')

# Format appearance
ax[0,0].set_ylim([-6.5,0.5])
ax[0,0].set_yticks(np.arange(-6,1,2))
ax[0,0].set_yticklabels([-6, '', '', 0])
ax[0,0].spines.left.set_bounds(-6, 0)
ax[0,0].spines.left.set_position(('outward', 5))
ax[0,0].spines.right.set_visible(False)
ax[0,0].spines.top.set_visible(False)
ax[0,0].yaxis.set_ticks_position('left')
ax[0,0].xaxis.set_ticks_position('bottom')

ax[0,1].set_ylim([-6.5,0.5])
ax[0,1].set_yticks(np.arange(-6,1,2))
ax[0,1].set_yticklabels([-6, '', '', 0])
ax[0,1].spines.left.set_bounds(-6, 0)
ax[0,1].spines.left.set_position(('outward', 5))
ax[0,1].spines.right.set_visible(False)
ax[0,1].spines.top.set_visible(False)
ax[0,1].yaxis.set_ticks_position('left')
ax[0,1].xaxis.set_ticks_position('bottom')

ax[1,0].set_ylim([-6e4,2e5])
ax[1,0].set_yticks(np.linspace(-5e4,1.9e5,4))
ax[1,0].set_yticklabels([])
ax[1,0].spines.left.set_bounds(-5e4,1.9e5)
ax[1,0].spines.left.set_position(('outward', 5))
ax[1,0].spines.right.set_visible(False)
ax[1,0].spines.top.set_visible(False)
ax[1,0].yaxis.set_ticks_position('left')
ax[1,0].xaxis.set_ticks_position('bottom')

ax[1,1].set_ylim([0,2.7e5])
ax[1,1].set_yticks(np.linspace(1e4,2.6e5,4))
ax[1,1].set_yticklabels([])
ax[1,1].spines.left.set_bounds(1e4,2.6e5)
ax[1,1].spines.left.set_position(('outward', 5))
ax[1,1].spines.right.set_visible(False)
ax[1,1].spines.top.set_visible(False)
ax[1,1].yaxis.set_ticks_position('left')
ax[1,1].xaxis.set_ticks_position('bottom')

ax[2,0].set_ylim([-0.1,1.2])
ax[2,0].set_yticks([0,1])
# ax[2,0].set_yticklabels(['Non- \n moving', 'Moving'])
ax[2,0].spines.left.set_bounds(0, 1)
ax[2,0].spines.left.set_position(('outward', 5))
ax[2,0].spines.right.set_visible(False)
ax[2,0].spines.top.set_visible(False)
ax[2,0].yaxis.set_ticks_position('left')
ax[2,0].xaxis.set_ticks_position('bottom')

ax[2,1].set_ylim([-0.1,1.2])
ax[2,1].set_yticks([0,1])
ax[2,1].spines.left.set_bounds(0, 1)
ax[2,1].spines.left.set_position(('outward', 5))
ax[2,1].spines.right.set_visible(False)
ax[2,1].spines.top.set_visible(False)
ax[2,1].yaxis.set_ticks_position('left')
ax[2,1].xaxis.set_ticks_position('bottom')

ax[3,0].set_ylim([1.5,2.3])
ax[3,0].set_yticks([1.6,2.2])
ax[3,0].spines.left.set_bounds(1.6,2.2)
ax[3,0].spines.left.set_position(('outward', 5))
ax[3,0].spines.right.set_visible(False)
ax[3,0].spines.top.set_visible(False)
ax[3,0].yaxis.set_ticks_position('left')
ax[3,0].xaxis.set_ticks_position('bottom')

ax[3,1].set_ylim([1.5,2.3])
ax[3,1].set_yticks([1.6,2.2])
ax[3,1].spines.left.set_bounds(1.6,2.2)
ax[3,1].spines.left.set_position(('outward', 5))
ax[3,1].spines.right.set_visible(False)
ax[3,1].spines.top.set_visible(False)
ax[3,1].yaxis.set_ticks_position('left')
ax[3,1].xaxis.set_ticks_position('bottom')

ax[0,0].set_xticks(np.arange(0,60*30,30))
ax[0,1].set_xlim([965,1175]) #session 4
ax[0,0].set_xticks(np.arange(965,965+211,30))
ax[3,1].set_xlabel('30sec');
for i in range(4):
    for j in range(2):
        ax[i,j].set_xticklabels([])
for i in range(4):
    ax[i,1].set_yticklabels([])

ax[2,1].legend(loc='lower left', bbox_to_anchor=(0.93, 0.6), 
               prop={'size': 8}, facecolor='white', framealpha=1)

plt.tight_layout()
ax[3,1].xaxis.set_label_coords(0.92, -0.25)
plt.show()

In [ ]:
## Minimum data provided with manuscript ##
left_accel_true = np.vstack([watch_left.loc[watch_left.session_id==session].timestamp.values-t0, 
                             watch_left.loc[watch_left.session_id==session].accel.values,
                             np.log10(watch_left.loc[watch_left.session_id==session].accel.values)>-3]).T
idx = (left_accel_true[:,0]>965) & (left_accel_true[:,0]<1175)
left_accel_true = left_accel_true[idx,:]
fig5b_left_accel_true = pd.DataFrame(left_accel_true, columns=['time','acceleration','state'])
# fig5b_left_accel_true.to_csv('fig5b_left_accel_true.csv', index=False)

right_accel_true = np.vstack([watch_right.loc[watch_right.session_id==session].timestamp.values-t0, 
                              watch_right.loc[watch_right.session_id==session].accel.values,
                              np.log10(watch_right.loc[watch_right.session_id==session].accel.values)>-3]).T
idx = (right_accel_true[:,0]>965) & (right_accel_true[:,0]<1175)
right_accel_true = right_accel_true[idx,:]
fig5b_right_accel_true = pd.DataFrame(right_accel_true, columns=['time','acceleration','state'])
# fig5b_right_accel_true.to_csv('fig5b_right_accel_true.csv', index=False)

left_accel_pred = np.vstack([rcs_right.loc[rcs_right.session_id==session].timestamp.values-t0, 
                             rcs_right.loc[rcs_right.session_id==session].output.values,
                             rcs_right.loc[rcs_right.session_id==session].state.values,
                             rcs_right.loc[rcs_right.session_id==session].stim.values]).T
idx = (left_accel_pred[:,0]>965) & (left_accel_pred[:,0]<1175)
left_accel_pred = left_accel_pred[idx,:]
fig5b_left_accel_pred = pd.DataFrame(left_accel_pred, columns=['time','acceleration','state','stim'])
# fig5b_left_accel_pred.to_csv('fig5b_left_accel_pred.csv', index=False)

right_accel_pred = np.vstack([rcs_left.loc[rcs_left.session_id==session].timestamp.values-t0, 
                              rcs_left.loc[rcs_left.session_id==session].output.values,
                              rcs_left.loc[rcs_left.session_id==session].state.values,
                              rcs_left.loc[rcs_left.session_id==session].stim.values]).T
idx = (right_accel_pred[:,0]>965) & (right_accel_pred[:,0]<1175)
right_accel_pred = right_accel_pred[idx,:]
fig5b_right_accel_pred = pd.DataFrame(right_accel_pred, columns=['time','acceleration','state','stim'])
# fig5b_right_accel_pred.to_csv('fig5b_right_accel_pred.csv', index=False)






# session = 4

# # Plot example segment
# plt.style.use('default')
# matplotlib.rcParams['font.sans-serif'] = "Arial"
# matplotlib.rcParams['font.family'] = "sans-serif"
# fig, ax = plt.subplots(4,2, figsize=(6.5,4.5), sharex=True)

# t0 = rcs_left.loc[rcs_left.session_id==session].timestamp.values[0]




# ax[0,0].set_title('Left Hemisphere')
# ax[0,0].plot(fig5b_right_accel_true['time'], 
#              np.clip(np.log10(fig5b_right_accel_true['acceleration']), -6, 0))
# ax[0,0].set_ylabel('True \n acceleration \n (Log-power)')
# ax[1,0].plot(fig5b_right_accel_pred['time'], 
#              fig5b_right_accel_pred['acceleration'], 
#              color='tab:orange')
# ax[1,0].set_ylabel('Predicted \n acceleration \n (Z-score)')


# ax[2,0].fill_between(fig5b_right_accel_true['time'], 
#                      fig5b_right_accel_true['state'] + 0.02, 
#                      alpha=0.5, step='mid', label='True', clip_on=False, zorder=10)
# ax[2,0].fill_between(fig5b_right_accel_pred['time'], 
#                      fig5b_right_accel_pred['state'] + 0.02, 
#                      alpha=0.5, step='mid', label='Predicted', clip_on=False, zorder=10)
# ax[2,0].set_ylabel('Movement \n state')
# ax[3,0].plot(fig5b_right_accel_pred['time'], 
#              fig5b_right_accel_pred['stim'], 
#              color='tab:orange')
# ax[3,0].set_ylabel('Stimulation \n (mA)')

# ax[0,1].set_title('Right Hemisphere')
# ax[0,1].plot(fig5b_left_accel_true['time'], 
#              np.clip(np.log10(fig5b_left_accel_true['acceleration']), -6, 0))
# ax[1,1].plot(fig5b_left_accel_pred['time'], 
#              fig5b_left_accel_pred['acceleration'], 
#              color='tab:orange')

# ax[2,1].fill_between(fig5b_left_accel_true['time'], 
#                      fig5b_left_accel_true['state'] + 0.02, 
#                      alpha=0.5, step='mid', label='True', clip_on=False, zorder=10)
# ax[2,1].fill_between(fig5b_left_accel_pred['time'], 
#                      fig5b_left_accel_pred['state'] + 0.02, 
#                      alpha=0.5, step='mid', label='Predicted', clip_on=False, zorder=10)
# ax[3,1].plot(fig5b_left_accel_pred['time'], 
#              fig5b_left_accel_pred['stim'], 
#              color='tab:orange')

# # Format appearance
# ax[0,0].set_ylim([-6.5,0.5])
# ax[0,0].set_yticks(np.arange(-6,1,2))
# ax[0,0].set_yticklabels([-6, '', '', 0])
# ax[0,0].spines.left.set_bounds(-6, 0)
# ax[0,0].spines.left.set_position(('outward', 5))
# ax[0,0].spines.right.set_visible(False)
# ax[0,0].spines.top.set_visible(False)
# ax[0,0].yaxis.set_ticks_position('left')
# ax[0,0].xaxis.set_ticks_position('bottom')

# ax[0,1].set_ylim([-6.5,0.5])
# ax[0,1].set_yticks(np.arange(-6,1,2))
# ax[0,1].set_yticklabels([-6, '', '', 0])
# ax[0,1].spines.left.set_bounds(-6, 0)
# ax[0,1].spines.left.set_position(('outward', 5))
# ax[0,1].spines.right.set_visible(False)
# ax[0,1].spines.top.set_visible(False)
# ax[0,1].yaxis.set_ticks_position('left')
# ax[0,1].xaxis.set_ticks_position('bottom')

# ax[1,0].set_ylim([-6e4,2e5])
# ax[1,0].set_yticks(np.linspace(-5e4,1.9e5,4))
# ax[1,0].set_yticklabels([])
# ax[1,0].spines.left.set_bounds(-5e4,1.9e5)
# ax[1,0].spines.left.set_position(('outward', 5))
# ax[1,0].spines.right.set_visible(False)
# ax[1,0].spines.top.set_visible(False)
# ax[1,0].yaxis.set_ticks_position('left')
# ax[1,0].xaxis.set_ticks_position('bottom')

# ax[1,1].set_ylim([0,2.7e5])
# ax[1,1].set_yticks(np.linspace(1e4,2.6e5,4))
# ax[1,1].set_yticklabels([])
# ax[1,1].spines.left.set_bounds(1e4,2.6e5)
# ax[1,1].spines.left.set_position(('outward', 5))
# ax[1,1].spines.right.set_visible(False)
# ax[1,1].spines.top.set_visible(False)
# ax[1,1].yaxis.set_ticks_position('left')
# ax[1,1].xaxis.set_ticks_position('bottom')

# ax[2,0].set_ylim([-0.1,1.2])
# ax[2,0].set_yticks([0,1])
# # ax[2,0].set_yticklabels(['Non- \n moving', 'Moving'])
# ax[2,0].spines.left.set_bounds(0, 1)
# ax[2,0].spines.left.set_position(('outward', 5))
# ax[2,0].spines.right.set_visible(False)
# ax[2,0].spines.top.set_visible(False)
# ax[2,0].yaxis.set_ticks_position('left')
# ax[2,0].xaxis.set_ticks_position('bottom')

# ax[2,1].set_ylim([-0.1,1.2])
# ax[2,1].set_yticks([0,1])
# ax[2,1].spines.left.set_bounds(0, 1)
# ax[2,1].spines.left.set_position(('outward', 5))
# ax[2,1].spines.right.set_visible(False)
# ax[2,1].spines.top.set_visible(False)
# ax[2,1].yaxis.set_ticks_position('left')
# ax[2,1].xaxis.set_ticks_position('bottom')

# ax[3,0].set_ylim([1.5,2.3])
# ax[3,0].set_yticks([1.6,2.2])
# ax[3,0].spines.left.set_bounds(1.6,2.2)
# ax[3,0].spines.left.set_position(('outward', 5))
# ax[3,0].spines.right.set_visible(False)
# ax[3,0].spines.top.set_visible(False)
# ax[3,0].yaxis.set_ticks_position('left')
# ax[3,0].xaxis.set_ticks_position('bottom')

# ax[3,1].set_ylim([1.5,2.3])
# ax[3,1].set_yticks([1.6,2.2])
# ax[3,1].spines.left.set_bounds(1.6,2.2)
# ax[3,1].spines.left.set_position(('outward', 5))
# ax[3,1].spines.right.set_visible(False)
# ax[3,1].spines.top.set_visible(False)
# ax[3,1].yaxis.set_ticks_position('left')
# ax[3,1].xaxis.set_ticks_position('bottom')

# ax[0,1].set_xlim([965,1175]) #session 4
# ax[0,0].set_xticks(np.arange(965,965+211,30))
# ax[3,1].set_xlabel('30sec');
# for i in range(4):
#     for j in range(2):
#         ax[i,j].set_xticklabels([])
# for i in range(4):
#     ax[i,1].set_yticklabels([])

# ax[2,1].legend(loc='lower left', bbox_to_anchor=(0.93, 0.6), 
#                prop={'size': 8}, facecolor='white', framealpha=1)

# plt.tight_layout()
# ax[3,1].xaxis.set_label_coords(0.92, -0.25)
# plt.show()

## Online classification performance

In [ ]:
arm_segment_names = ['chest_tapping', 
                     'rest', 
                     'finger_tapping_right',
                     'finger_tapping_left',
                     'wrist_rotation_right',
                     'wrist_rotation_left',
                     'nose_tapping_right',
                     'nose_tapping_left']

### Figure 5c, Supplementary Figure 2: Online classifier performance (confusion matrices)

In [ ]:
num_sessions = 12

acc_left = np.zeros([num_sessions, 4])
f1_left = np.zeros([num_sessions, 4])
balacc_left = np.zeros([num_sessions, 4])
conf_mat_left = [[0 for i in range(4)] for s in range(num_sessions)]
mean_stim_left = np.zeros([num_sessions, 4]) # currently only fills the first three columns
acc_right = np.zeros([num_sessions, 4])
f1_right = np.zeros([num_sessions, 4])
balacc_right = np.zeros([num_sessions, 4])
conf_mat_right = [[0 for i in range(4)] for s in range(num_sessions)]
mean_stim_right = np.zeros([num_sessions, 4]) # currently only fills the first three columns

n_pred_left = 0
n_true_left = 0
n_correct_left = 0
n_total_left = 0
n_pred_right = 0
n_true_right = 0
n_correct_right = 0
n_total_right = 0

column_lbls = [f"{a}_{b}" for a, b in 
               product(['movement_responsive','inverted','constant'], 
                       ['TP','FP','TN','FN'])]

# session=12
conf_mat_left_1d = [np.array([]) for i in range(12)]
conf_mat_right_1d = [np.array([]) for i in range(12)]
for s in range(num_sessions):
    for i in range(4):
        acc_left[s,i], f1_left[s,i], conf_mat_left[s][i] = \
            compute_online_performance(watch_right.loc[watch_right.session_id==(s+1)], 
                                       rcs_left.loc[rcs_left.session_id==(s+1)], 
                                       behavior_segments=arm_segment_names, 
                                       config_block=i+1, lag=-0.3)
        balacc_left[s,i] = 0.5*(conf_mat_left[s][i]['TP']/(conf_mat_left[s][i]['TP']+conf_mat_left[s][i]['FN'])
                                 + conf_mat_left[s][i]['TN']/(conf_mat_left[s][i]['TN']+conf_mat_left[s][i]['FP']))
        
        if i<3:
            conf_mat_left_1d[s] = np.hstack([conf_mat_left_1d[s], 
                                             list(conf_mat_left[s][i].values())])
        conf_mat_left[s][i] = \
            np.array([[conf_mat_left[s][i]['TN'], conf_mat_left[s][i]['FP']],
                      [conf_mat_left[s][i]['FN'], conf_mat_left[s][i]['TP']]])
        n_pred_left += (conf_mat_left[s][i][0,1] + conf_mat_left[s][i][1,1])
        n_true_left += (conf_mat_left[s][i][1,0] + conf_mat_left[s][i][1,1])
        n_total_left += np.sum(conf_mat_left[s][i])
        n_correct_left += (conf_mat_left[s][i][0,0] + conf_mat_left[s][i][1,1])
        conf_mat_left[s][i] = np.around(conf_mat_left[s][i] /
                                     np.tile(np.sum(conf_mat_left[s][i], 1, 
                                                    keepdims=True), 2),
                                     decimals=2)
        
        if i<3:
            mean_stim_left[s,i] = np.mean(rcs_left.loc[(rcs_left.session_id==(s+1)) 
                                                       & (rcs_left.block_id==(i+1))].stim)
        

        acc_right[s,i], f1_right[s,i], conf_mat_right[s][i] = \
            compute_online_performance(watch_left.loc[watch_left.session_id==(s+1)], 
                                       rcs_right.loc[rcs_right.session_id==(s+1)], 
                                       behavior_segments=arm_segment_names, 
                                       config_block=i+1, lag=-0.3)
        balacc_right[s,i] = 0.5*(conf_mat_right[s][i]['TP']/(conf_mat_right[s][i]['TP']+conf_mat_right[s][i]['FN'])
                                 + conf_mat_right[s][i]['TN']/(conf_mat_right[s][i]['TN']+conf_mat_right[s][i]['FP']))
        
        if i<3:
            conf_mat_right_1d[s] = np.hstack([conf_mat_right_1d[s], 
                                             list(conf_mat_right[s][i].values())])
        conf_mat_right[s][i] = \
            np.array([[conf_mat_right[s][i]['TN'], conf_mat_right[s][i]['FP']],
                      [conf_mat_right[s][i]['FN'], conf_mat_right[s][i]['TP']]])
        n_pred_right += (conf_mat_right[s][i][0,1] + conf_mat_right[s][i][1,1])
        n_true_right += (conf_mat_right[s][i][1,0] + conf_mat_right[s][i][1,1])
        n_total_right += np.sum(conf_mat_right[s][i])
        n_correct_right += (conf_mat_right[s][i][0,0] + conf_mat_right[s][i][1,1])
        conf_mat_right[s][i] = np.around(conf_mat_right[s][i] /
                                      np.tile(np.sum(conf_mat_right[s][i], 1, 
                                                     keepdims=True), 2),
                                      decimals=2)
        
        if i<3:
            mean_stim_right[s,i] = np.mean(rcs_right.loc[(rcs_right.session_id==(s+1)) 
                                                       & (rcs_right.block_id==(i+1))].stim)

conf_mat_left_full = np.sum(conf_mat_left, axis=0)
conf_mat_left_full_ppn = np.zeros(np.shape(conf_mat_left_full))
conf_mat_right_full = np.sum(conf_mat_right, axis=0)
conf_mat_right_full_ppn = np.zeros(np.shape(conf_mat_right_full))
for i in range(4):
    conf_mat_left_full_ppn[i,:,:] = \
        np.around(conf_mat_left_full[i,:,:] / \
                  np.tile(np.sum(conf_mat_left_full[i,:,:], 1, keepdims=True),
                          2), decimals=2)
    conf_mat_right_full_ppn[i,:,:] = \
        np.around(conf_mat_right_full[i,:,:] / \
                  np.tile(np.sum(conf_mat_right_full[i,:,:], 1, keepdims=True),
                          2), decimals=2)

## Minimum dataset provided with manuscript ##
conf_mat_left_1d = np.array(conf_mat_left_1d)
fig5c_conf_mat_left = pd.DataFrame(conf_mat_left_1d, columns=column_lbls)
# fig5c_conf_mat_left.to_csv('fig5c_conf_mat_left.csv', index=False)
conf_mat_right_1d = np.array(conf_mat_right_1d)
fig5c_conf_mat_right = pd.DataFrame(conf_mat_right_1d, columns=column_lbls)
# fig5c_conf_mat_right.to_csv('fig5c_conf_mat_right.csv', index=False)

In [ ]:
p_left = binomtest(n_correct_left, n_total_left, p=0.5)
p_right = binomtest(n_correct_right, n_total_right, p=0.5)

print('LEFT HEMISPHERE')
print('Mean accuracy: ' + str(np.mean(acc_left, axis=0)[3]))
print('Accuracy>0.5 p-value: ' + str(p_left.pvalue))
print('Mean F1 score: ' + str(np.mean(f1_left, axis=0)[3]))
print('Mean balanced accuracy: ' + str(np.mean(balacc_left, axis=0)[3]))
print('Proportion true movement state:' + str(n_true_left/n_total_left))
print('Prediction bias: ' + str(n_pred_left/n_total_left
                                - n_true_left/n_total_left))

print('\n')

print('RIGHT HEMISPHERE')
print('Mean accuracy: ' + str(np.mean(acc_right, axis=0)[3]))
print('Accuracy>0.5 p-value: ' + str(p_right.pvalue))
print('Mean F1 score: ' + str(np.mean(f1_right, axis=0)[3]))
print('Mean balanced accuracy: ' + str(np.mean(balacc_right, axis=0)[3]))
print('Proportion true movement state:' + str(n_true_right/n_total_right))
print('Prediction bias: ' + str(n_pred_right/n_total_right
                                - n_true_right/n_total_right))

In [ ]:
session = 12

fig = plt.figure(figsize=(6, 5))
fig.suptitle('Left hemisphere', fontsize=20)
gs = gridspec.GridSpec(nrows=2, ncols=3, height_ratios=[1.5, 1])

ax = [[] for i in range(4)]
ax[0] = fig.add_subplot(gs[0,:])
ax[0].set_aspect('equal')
df = pd.DataFrame(conf_mat_left_full_ppn[3], index=['Stationary', 'Moving'], 
                  columns=['Stationary', 'Moving'])
sns.heatmap(df, ax=ax[0], annot=True, vmin=0, vmax=1, cmap='coolwarm', 
            cbar_kws={"ticks": [0,1]})
ax[0].set_xlabel('LD0', fontsize=14)
ax[0].set_ylabel('Watch', fontsize=14)
ax[0].title.set_text('Combined')
ax[0].title.set_fontsize(18)

sub_titles = ['Test condition', 'Stim inversion', 'Open-loop 2mA']
for config in range(3):
    ax[config] = fig.add_subplot(gs[1,config])
    ax[config].set_aspect('equal')
    sns.heatmap(conf_mat_left_full_ppn[config], ax=ax[config], annot=True, vmin=0, vmax=1, 
                cmap='coolwarm', cbar=False, xticklabels=False, 
                yticklabels=False)
    ax[config].title.set_text(sub_titles[config])

plt.tight_layout(pad=2.0)

In [ ]:
fig = plt.figure(figsize=(6, 5))
fig.suptitle('Right hemisphere', fontsize=20)
gs = gridspec.GridSpec(nrows=2, ncols=3, height_ratios=[1.5, 1])

ax = [[] for i in range(4)]
ax[0] = fig.add_subplot(gs[0,:])
ax[0].set_aspect('equal')
df = pd.DataFrame(conf_mat_right_full_ppn[3], index=['Stationary', 'Moving'], 
                  columns=['Stationary', 'Moving'])
sns.heatmap(df, ax=ax[0], annot=True, vmin=0, vmax=1, cmap='coolwarm',
           cbar_kws={"ticks": [0,1]})
ax[0].set_xlabel('LD0', fontsize=14)
ax[0].set_ylabel('Watch', fontsize=14)
ax[0].title.set_text('Combined')
ax[0].title.set_fontsize(18)

sub_titles = ['Test condition', 'Stim inversion', 'Open-loop 2mA']
for config in range(3):
    ax[config] = fig.add_subplot(gs[1,config])
    ax[config].set_aspect('equal')
    sns.heatmap(conf_mat_right_full_ppn[config], ax=ax[config], annot=True, vmin=0, vmax=1, 
                cmap='coolwarm', cbar=False, xticklabels=False, 
                yticklabels=False)
    ax[config].title.set_text(sub_titles[config])

plt.tight_layout(pad=2.0)

### Figure 5d: Detection latencies

In [ ]:
def find_single_latency(rcs, watch, hand, session_id, block_id, segment_id):
    """
    Compute latency of movement state classification for a single movement.

    Parameters
    ----------
    rcs : DataFrame
        Pandas dataframe containing neural data and task data
    watch : DataFrame
        Pandas dataframe containing movement data
    hand : string
        The hand to analyze: 'left' or 'right'
    session_id : int [1:12]
        The session to analyze
    session_id : int [1:12]
        The session to analyze
    block_id : int [1:3]
        The block to analyze. 1: Movement-responsive, 2: Inverted, 3: Constant
    segment_id : string
        The movement to analyze. Options: 'chest_tapping', 'rest', 
            'finger_tapping_right', 'finger_tapping_left',   
            'wrist_rotation_right', 'wrist_rotation_left', 
            'nose_tapping_right', 'nose_tapping_left', 'typing'
    
    Returns
    -------
    latency : (2,) array of floats
        The movement state classification latency for the onset and offset of
            the selected movement. Units in sec.
    state : (2,n) array
        The timestamps (in sec) and movement states within the window of 
            interest
    pred : (2,m) array
        The timestamps (in sec) and state predictions within the window of 
            interest

    """
    
    
    # Select the chosen movement window with 3 second buffer on either end
    segment_idx_start = np.where((rcs.session_id==session_id) 
                                 & (rcs.block_id==block_id) 
                                 & (rcs.segment_id==segment_id))[0][0]
    segment_idx_end = np.where((rcs.session_id==session_id) 
                               & (rcs.block_id==block_id) 
                               & (rcs.segment_id==segment_id))[0][-1]
    ts_start = rcs.timestamp.values[segment_idx_start] - 3
    ts_end = rcs.timestamp.values[segment_idx_end] + 3
    
    rcs_idx_start = np.where(rcs.timestamp>ts_start)[0][0]
    rcs_idx_end = np.where(rcs.timestamp>ts_end)[0][0]
    rcs = rcs[rcs_idx_start:rcs_idx_end]
    
    watch_idx_start = np.where(watch.timestamp>ts_start)[0][0]
    watch_idx_end = np.where(watch.timestamp>ts_end)[0][0]
    watch = watch[watch_idx_start:watch_idx_end]
    
    if segment_id in ['chest_tapping', 'rest']:
        task_duration = 5
    elif segment_id in ['finger_tapping_right', 'finger_tapping_left',   
                      'wrist_rotation_right', 'wrist_rotation_left', 
                      'nose_tapping_right', 'nose_tapping_left']:
        task_duration = 15
    elif segment_id == 'typing':
        task_duration = 20
    
    
    # Determine the movement state of this hand and task segment (moving or not)
    left_mvmts = ['chest_tapping', 'finger_tapping_left', 
                  'wrist_rotation_left', 'nose_tapping_left', 'typing']
    left_nonmvmts = ['rest', 'finger_tapping_right', 'wrist_rotation_right', 
                     'nose_tapping_right']
    right_mvmts = ['chest_tapping', 'finger_tapping_right', 
                   'wrist_rotation_right', 'nose_tapping_right', 'typing']
    right_nonmvmts = ['rest', 'finger_tapping_left', 'wrist_rotation_left', 
                      'nose_tapping_left']
    if hand=='left':
        if segment_id in left_mvmts:
            move_state = 1
        elif segment_id in left_nonmvmts:
            move_state = 0
        else:
            raise Exception("Invalid segment_id input")
    elif hand=='right':
        if segment_id in right_mvmts:
            move_state = 1
        elif segment_id in right_nonmvmts:
            move_state = 0
        else:
            raise Exception("Invalid segment_id input")
    else:
        raise Exception("Invalid hand input")
        
        
    # Find the edges of the movement state (move->non, and non->move)
    steady_state_move = np.zeros(np.shape(watch.state.values)).astype(int)
    steady_state_non = np.zeros(np.shape(watch.state.values)).astype(int)
    for i in range(10): # 5 seconds of the same state
            steady_state_move += np.roll(watch.state.values,-i)
            steady_state_non += np.roll(np.abs(watch.state.values-1),-i)
    if move_state: # if a task segment where this hand is supposed to be moving
        idx_state_start = np.where(steady_state_move==10)[0][0]
        ts_state_start = watch.timestamp.values[idx_state_start]
        valid_end_samples = sum((steady_state_non==10) 
                                & (watch.timestamp.values>ts_state_start+task_duration))>0
        if valid_end_samples:
            idx_state_end = np.where((steady_state_non==10) 
                & (watch.timestamp.values>ts_state_start+task_duration))[0][0]
        else:
            idx_state_end = []
    else: # if a task segment where this hand is not supposed to be moving
        idx_state_start = np.where(steady_state_non==10)[0][0]
        ts_state_start = watch.timestamp.values[idx_state_start]
        valid_end_samples = sum((steady_state_move==10) 
                                & (watch.timestamp.values>ts_state_start+task_duration))>0
        if valid_end_samples:
            idx_state_end = np.where((steady_state_move==10) 
                & (watch.timestamp.values>ts_state_start+task_duration))[0][0]
        else:
            idx_state_end = []
    ts_state_end = watch.timestamp.values[idx_state_end]
    if ts_state_end.size==0:
        ts_state_end = None
        
        
    # Find the edges of the state predictions
    idx_start_candidates = rcs.timestamp.values-ts_state_start > -2
    if idx_state_end: # if there was a valid end of the state detected
        idx_end_candidates = rcs.timestamp.values-ts_state_end > -2
    else:
        idx_end_candidates = np.full(rcs.timestamp.values.shape, False)
    if move_state: # if a task segment where this hand is supposed to be moving
        valid_start_samples = np.sum(idx_start_candidates 
                                     & (rcs.state.values==1))>0
        if valid_start_samples:
            idx_pred_start = np.where(idx_start_candidates 
                                      & (rcs.state.values==1))[0][0]
        else:
            idx_pred_start = []
        valid_end_samples = np.sum(idx_end_candidates 
                                   & (rcs.state.values==0))>0
        if valid_end_samples:
            idx_pred_end = np.where(idx_end_candidates 
                                    & (rcs.state.values==0))[0][0]
        else:
            idx_pred_end = []

    else: # if a task segment where this hand is not supposed to be moving
        valid_start_samples = np.sum(idx_start_candidates 
                                     & (rcs.state.values==0))>0
        if valid_start_samples:
            idx_pred_start = np.where(idx_start_candidates 
                                      & (rcs.state.values==0))[0][0]
        else:
            idx_pred_start = []
        valid_end_samples = np.sum(idx_end_candidates 
                                   & (rcs.state.values==1))>0
        if valid_end_samples:
            idx_pred_end = np.where(idx_end_candidates 
                                    & (rcs.state.values==1))[0][0]
        else:
            idx_pred_end = []
    if idx_pred_start:
        ts_pred_start = rcs.timestamp.values[idx_pred_start]
    else:
        ts_pred_start = None
    if idx_pred_end:
        ts_pred_end = rcs.timestamp.values[idx_pred_end]
    else:
        ts_pred_end = None
    
    
    # Compute latencies
    if (ts_pred_start!=None) & (ts_state_start!=None):
        start_latency = ts_pred_start-ts_state_start
    else:
        start_latency = None
    if (ts_pred_end!=None) & (ts_state_end!=None):
        end_latency = ts_pred_end-ts_state_end
    else:
        end_latency = None
    latency = np.array([start_latency, end_latency])
    
    
    # Log the data from the time window of interest
    state = np.array([watch.timestamp.values, watch.state.values])
    pred = np.array([rcs.timestamp.values, rcs.state.values])
    ts = np.array([[ts_state_start, ts_state_end], 
                   [ts_pred_start, ts_pred_end]], dtype=object)
    
    
    return latency, state, pred, ts


def collect_latencies(latency_df, hand, segment_id):
    latency_df = latency_df.loc[(latency_df.segment_id.values==segment_id) 
                                & (latency_df.hand.values==hand)]
    latency_array = np.array([])
    for latency_vals in latency_df.latency.values:
        if len(latency_vals)>0:
            if latency_vals[0]:
                latency_array = np.append(latency_array, latency_vals[0])
                
    return latency_array

In [ ]:
watch_left['state'] = (np.log10(watch_left.accel.values)>-3).astype(int)
watch_right['state'] = (np.log10(watch_right.accel.values)>-3).astype(int)

latency_df = pd.DataFrame(columns=['session_id', 'block_id', 'segment_id', 
                                   'hand', 'latency', 'state_data', 
                                   'prediction_data', 'transition_ts'])
segment_id_list = ['chest_tapping', 'rest', 
                   'finger_tapping_right', 'finger_tapping_left',   
                   'wrist_rotation_right', 'wrist_rotation_left', 
                   'nose_tapping_right', 'nose_tapping_left', 'typing']
for session_id in range(12):
    for block_id in range(3):
        for segment_id in segment_id_list:
            for hand in ['left', 'right']:
                if hand == 'left':
                    is_valid = np.sum((rcs_right.session_id==session_id+1) 
                                      & (rcs_right.block_id==block_id+1) 
                                      & (rcs_right.segment_id==segment_id))>0
                    if is_valid:
                        latency, state, pred, ts = find_single_latency(
                            rcs_right, watch_left, hand, 
                            session_id+1, block_id+1, segment_id)
                    else:
                        latency = []
                        state = []
                        pred = []
                        ts = []
                else:
                    is_valid = np.sum((rcs_left.session_id==session_id+1) 
                                      & (rcs_left.block_id==block_id+1) 
                                      & (rcs_left.segment_id==segment_id))>0
                    if is_valid:
                        latency, state, pred, ts = find_single_latency(
                            rcs_left, watch_right, hand, 
                            session_id+1, block_id+1, segment_id)
                    else:
                        latency = []
                        state = []
                        pred = []
                        ts = []
                new_row = {'session_id': session_id, 'block_id': block_id, 
                           'segment_id': segment_id, 'hand': hand, 
                           'latency': latency, 'state_data': state, 
                           'prediction_data': pred, 'transition_ts': ts}
                latency_df = pd.concat([latency_df, pd.DataFrame([new_row])], 
                                       ignore_index=True)

In [ ]:
segment_id_list = ['finger_tapping', 'wrist_rotation', 'nose_tapping', 'typing']
left_hem_combined_latencies = np.array([])
right_hem_combined_latencies = np.array([])

fig, ax = plt.subplots(1,1, figsize=(4.2,1.8))
ax.fill_betweenx([-2, 6], [-2,-2], [4,4], 
                 facecolor='grey', edgecolor=None, alpha=0.2)
plt_colors = np.array([[255, 89, 94],
                       [138, 201, 38],
                       [25, 130, 196],
                       [106, 76, 147]])/255

splots = [0 for i in range(9)]
bplots = [0 for i in range(9)]
for h,hand in enumerate(['right', 'left']):
    for s,segment_id in enumerate(segment_id_list):
        x = h*5 + s
        if segment_id != 'typing':
            segment_id += '_' + hand
        latency_array = collect_latencies(latency_df, hand, segment_id)
        latency_array = latency_array[(latency_array>-1.5)&(latency_array<5)]
        splots[h*5+s] = ax.plot(x*np.ones(len(latency_array)), latency_array, 
                                '.', color=plt_colors[s])
        # np.savetxt('fig5d_latency_' + hand + '_' + segment_id + '.csv', 
        #            splots[h*5+s][0].get_ydata(), delimiter=',')
        bplots[h*5+s] = ax.boxplot(latency_array, positions=[x], widths=0.4,
                                   showfliers=False, patch_artist=True, 
                                   boxprops=dict(facecolor=plt_colors[s], alpha=0.5), 
                                   medianprops={'linewidth': 2, 'color': 'k'})
        if hand == 'right':
            left_hem_combined_latencies = \
                np.concatenate([left_hem_combined_latencies, latency_array])
        else:
            right_hem_combined_latencies = \
                np.concatenate([right_hem_combined_latencies, latency_array])
            
        
xticklabels = ['Finger taps', 'Wrist rotations', 'Nose taps', 'Typing']*2        
ax.set_xticklabels(xticklabels, rotation=45, ha='right', fontsize=6)
ax.set_xlim(-1, 9)
ax.tick_params(axis='y', right=True, labelsize=5)
ax.set_yticks([-2, 0, 2, 4, 6])
ax.set_ylabel('Detection latency (s)', fontsize=6)
ax.set_ylim(-2, 6)
ax.annotate('Left Hemisphere', xy=(1.5, 5.4), xytext=(1.5, 5.1), 
            ha="center", fontsize=7)
ax.annotate('Right Hemisphere', xy=(7.5, 5.4), xytext=(6.5, 5.1), 
            ha="center", fontsize=7)

plt.tight_layout()
# plt.savefig('latencies.svg', format='svg')

In [ ]:
print('Left hemisphere mean overall detection latency:')
print('\t'+str(np.mean(left_hem_combined_latencies)))
print('Right hemisphere mean overall detection latency:')
print('\t'+str(np.mean(right_hem_combined_latencies)))

### Figure 5e: Distribution of stimulation amplitudes in each movement state

In [ ]:
stim_dist_left = [0 for i in range(12)]
stim_dist_right = [0 for i in range(12)]

for i in range(12):
    stim_dist_left[i] = compute_stim_distribution(watch_right, rcs_left, 
                                               stim_lvls=[1.6, 1.9, 2.2], 
                                               behavior_segments=arm_segment_names, 
                                               return_ppn=True, lag=-0.3)
    stim_dist_right[i] = compute_stim_distribution(watch_left, rcs_right, 
                                                stim_lvls=[1.6, 1.9, 2.2], 
                                                behavior_segments=arm_segment_names, 
                                                return_ppn=True, lag=-0.3)
    
stim_dist_left = np.mean(stim_dist_left,0)
stim_dist_right = np.mean(stim_dist_right,0)

In [ ]:
plt.style.use('ggplot')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
fig, ax = plt.subplots(1,2, figsize=(6,2.2), sharex='row', sharey='row')

ax[0].set_title('Left Hemisphere')
ax[1].set_title('Right Hemisphere')

for i,b in enumerate([2,0,1]):
    ax[0].bar([0+2*i,0.6+2*i], stim_dist_left[b][0,:], 
              color='tab:blue', edgecolor='white', alpha=0.9, width=0.5)
    ax[0].bar([0+2*i,0.6+2*i], 1-np.sum(stim_dist_left[b][[0,2],:], 0), 
              bottom=stim_dist_left[b][0,:], 
              color='tab:grey', edgecolor='white', alpha=0.9, width=0.5)
    ax[0].bar([0+2*i,0.6+2*i], stim_dist_left[b][2,:], 
              bottom=1-stim_dist_left[b][2,:], 
              color='tab:green', edgecolor='white', alpha=0.9, width=0.5)
    
    ax[1].bar([0+2*i,0.6+2*i], stim_dist_right[b][0,:], 
              color='tab:blue', edgecolor='white', alpha=0.9, width=0.5)
    ax[1].bar([0+2*i,0.6+2*i], 1-np.sum(stim_dist_right[b][[0,2],:], 0), 
              bottom=stim_dist_right[b][0,:], 
              color='tab:grey', edgecolor='white', alpha=0.9, width=0.5)
    ax[1].bar([0+2*i,0.6+2*i], stim_dist_right[b][2,:], 
              bottom=1-stim_dist_right[b][2,:], 
              color='tab:green', edgecolor='white', alpha=0.9, width=0.5)
ax[0].set_ylabel('Proportion')

upper_ax = [ax[0].twiny(), ax[1].twiny()]
for i in range(2):
    ax[i].set_xlim([-0.5, 5.1])
    ax[i].set_xticks([0, 0.6, 2, 2.6, 4, 4.6])
    ax[i].tick_params(bottom=False)
    ax[i].set_xticklabels(['Stationary', 'Moving']*3, rotation=45, ha='right', 
                          rotation_mode='anchor', fontsize=8)
    ax[i].grid(visible=False, axis='x')
    ax[i].tick_params(axis='x', which='major', pad=0)
    ax[i].set_yticks(np.arange(0,1.1,0.25))
    ax[i].set_yticklabels(['0', '', '', '', '1'])

    upper_ax[i].set_xlim([-0.5, 5.1])
    upper_ax[i].set_xticks([0.3, 2.3, 4.3])
    upper_ax[i].tick_params(top=False)
    upper_ax[i].set_xticklabels(['Continuous', 'Movement\nresponsive', 'Inverted'], 
                                fontsize=9)
    upper_ax[i].grid(visible=False, axis='x')
    upper_ax[i].tick_params(axis='x', which='major', pad=0)
    upper_ax[i].spines.right.set_visible(False)


leg = ax[1].legend(['Low stim', 'Middle stim', 'High stim'], loc='lower left', 
                   bbox_to_anchor=(0.95, -0.05), prop={'size': 8}, 
                   facecolor='white', framealpha=1)

plt.tight_layout()

In [ ]:
## Minimum data provided with manuscript ##

column_lbls = ['left_stat_mr','left_move_mr','left_stat_inv','left_move_inv',
               'right_stat_mr','right_move_mr','right_stat_inv','right_move_inv']
row_lbls = ['low_stim','high_stim']

fig5e_stim_x_state = np.hstack([stim_dist_left[c][[0,2],:] for c in range(2)] +
                                [stim_dist_right[c][[0,2],:] for c in range(2)])
fig5e_stim_x_state = pd.DataFrame(fig5e_stim_x_state, columns=column_lbls, index=row_lbls)
# fig5e_stim_x_state.to_csv('fig5e_stim_x_state.csv')

## Therapeutic evaluation

### Figure 6a: Comparison of blinded self-scores across stimulation conditions

In [ ]:
data = pd.DataFrame(data={'session_id' : np.repeat(np.arange(12), 3),
                          'config': np.tile([1,2,3], 12).astype(str), 
                          'score': self_score.to_numpy().flatten(),
                          'is_first': [1, 0, 0, 
                                       1, 0, 0, 
                                       0, 0, 1, 
                                       0, 0, 1,
                                       1, 0, 0, 
                                       0, 1, 0, 
                                       0, 1, 0,
                                       0, 1, 0,
                                       0, 0, 1, 
                                       0, 1, 0, 
                                       1, 0, 0, 
                                       0, 0, 1],
                          'is_second': [0, 1, 0, 
                                        0, 0, 1, 
                                        1, 0, 0, 
                                        0, 1, 0,
                                        0, 0, 1, 
                                        1, 0, 0, 
                                        1, 0, 0, 
                                        0, 0, 1,
                                        0, 1, 0, 
                                        0, 0, 1, 
                                        0, 1, 0, 
                                        1, 0, 0]})
data['score_adjusted'] = data['score']
for session_id in np.unique(data.session_id):
    mean_score = np.mean(data[data.session_id==session_id].score)
    data.loc[data.session_id==session_id, 'score_adjusted'] -= mean_score

# Fit the one-way ANCOVA model
self_score_mdl = sm.OLS.from_formula('score_adjusted ~ config + is_first + is_second', data=data).fit()
self_score_anova_table = sm.stats.anova_lm(self_score_mdl, typ=2)

# Print the ANOVA table
print('BLINDED SELF-SCORE: One-way ANCOVA')
print(self_score_anova_table)

In [ ]:
self_score_mdl = sm.OLS.from_formula('score_adjusted ~ config + is_first', data=data).fit()

# Print the ANCOVA coefficients after including only significant terms
print('BLINDED SELF-SCORE: ANCOVA significant coefficients')
print(self_score_mdl.params)

In [ ]:
# control for the significant covariate effects
data['score_cov_adjusted'] = data.score_adjusted \
                             - self_score_mdl.params['is_first']*data.is_first

ttest_press_duration_1_2 = ttest_ind(data.score_cov_adjusted[(data.config.values=='1')], 
                                     data.score_cov_adjusted[(data.config.values=='2')])
ttest_press_duration_1_3 = ttest_ind(data.score_cov_adjusted[(data.config.values=='1')], 
                                     data.score_cov_adjusted[(data.config.values=='3')])
ttest_press_duration_2_3 = ttest_ind(data.score_cov_adjusted[(data.config.values=='2')], 
                                     data.score_cov_adjusted[(data.config.values=='3')])

print('PLANNED CONTRASTS \n Covariate-adjusted, exclude non-sig interaction')

print('\n BLINDED SELF-SCORE')
print('\t Stim condition 1 vs 2: ', ttest_press_duration_1_2)
print('\t Stim condition 1 vs 3: ', ttest_press_duration_1_3)
print('\t Stim condition 2 vs 3: ', ttest_press_duration_2_3)

## Minimum dataset provided with manuscript ##
# data.to_csv('fig6a_self_score.csv', index=False)

In [ ]:
plt.style.use('ggplot')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"

# self_score dataframe should contain columns: 
# session_id, main_adbs_score, inv_adbs_score, constant_score

fig, ax = plt.subplots(1,1, figsize=(3.1,2.1))
ax.add_patch(Rectangle((-1.5,10), 11.5, 1, color='w'))

data = data.astype({'config': int})
order = np.zeros(10, int)
order[[0,4,8]] = [3,1,2]
sns.swarmplot(data=data, 
              x='config', y='score', 
              order=np.roll(order, 1), alpha=0.8, size=4, hue='config', ax=ax)
sns.boxplot(data=data, 
            x='config', y='score', 
            order=order, dodge=False, hue='config', width=1, whis=10, ax=ax)

# styling
ax.set_ylabel('aDBS self-score \n (0-10)')
ax.set_xlabel('')
ax.set_xlim([-1.5, 10])
ax.set_xticks([0.4, 4.4, 8.4])
ax.set_xticklabels(['Constant', 'Movement \n responsive ', 'Inverted'])

ax.set_ylim([3, 11])
ax.set_yticks([4, 5, 6, 7, 8, 9, 10])
ax.set_yticklabels(['4', '', '', '', '', '', '10'])
    
# annotate to show significance - keypress duration
max_score_constant = np.max(data.score[(data.config.values==3)])
max_score_responsive = np.max(data.score[(data.config.values==1)])
max_score_inverted = np.max(data.score[(data.config.values==2)])

ax.axhline(y=10.9, xmin=1.9/11.5, xmax=9.9/11.5, color='k', linewidth=1)
ax.axhline(y=10.4, xmin=5.9/11.5, xmax=9.9/11.5, color='k', linewidth=1)
ax.axvline(x=0.4, 
              ymin=(max_score_constant+0.3-3)/8, 
              ymax=(10.9-3)/8, 
              color='k', linewidth=1)
ax.axvline(x=4.4, 
              ymin=(max_score_responsive+0.3-3)/8, 
              ymax=(10.4-3)/8, 
              color='k', linewidth=1)
ax.axvline(x=8.4, 
              ymin=(max_score_inverted+0.3-3)/8, 
              ymax=(10.9-3)/8,
              color='k', linewidth=1)
ax.annotate('*', xy=[4.4, 10.7])
ax.annotate('*', xy=[6.4, 10.2])

ax.get_legend().remove()
plt.tight_layout()

### Figure 6b: Correlation between self scores and performance (self score vs F1 score)

In [ ]:
f1 = (f1_left[:,0] + f1_right[:,0]) / 2

f1_self_score_corr, p_f1_self_score = pearsonr(f1, 
                                               self_score.main_adbs_score.values,
                                               alternative='greater')

In [ ]:
plt.style.use('ggplot')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
fig, ax = plt.subplots(1,1, figsize=(3.2,2.5), sharex='row', sharey='row')

ax.set_title('Self score vs Decoder F1 score  ')
ax.add_patch(Rectangle((0,10), 1, 1, color='w'))

z = np.polyfit(f1, self_score.main_adbs_score.values, 1)
p = np.poly1d(z)
ax.scatter(f1, self_score.main_adbs_score.values)
ax.plot(f1, p(f1))

ax.set_ylim([4.8, 10.2])
ax.set_yticks([5,6,7,8,9,10])
ax.set_yticklabels([5,'','','','',10])
ax.set_ylabel('aDBS rating \n (0-10)  ')
ax.set_xlim([0.65, 0.85])
ax.set_xticks(np.arange(0.65,0.9,0.05))
ax.set_xticklabels([0.65,'','','',0.85])
ax.set_xlabel('F1 score \n (sides combined)')

ax.annotate('r = ' + str(np.round(f1_self_score_corr,2)) 
            + '\np = ' + str(np.round(p_f1_self_score,3)), 
            xy=(0.66, 9))

plt.tight_layout()
# np.savetxt('fig6b_f1.csv', f1, delimiter=',')

### Figure 6c: Correlation between block-averaged stimulation amplitude and self scores (self score vs mean stim amplitude)

In [ ]:
mean_stim = (mean_stim_left + mean_stim_right) / 2

r_main, p_main = pearsonr(mean_stim[:,0], self_score.main_adbs_score.values)
r_inv, p_inv = pearsonr(mean_stim[:,1], self_score.inv_adbs_score.values)

print('Correlation between mean stim and self-score: Movement responsive')
print('\tp=' + str(p_main))
print('\tr=' + str(r_main))
print('\nCorrelation between mean stim and self-score: Inverted')
print('\tp=' + str(p_inv))
print('\tr=' + str(r_inv))

In [ ]:
plt.style.use('ggplot')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
fig, ax = plt.subplots(1,1, figsize=(4.2,2.5), sharex='row', sharey='row')

ax.set_title('Self score vs Mean stimulation')
ax.add_patch(Rectangle((0,10), 3, 1, color='w', label='_nolegend_'))

z_main = np.polyfit(mean_stim[:,0], self_score.main_adbs_score.values, 1)
f_main = np.poly1d(z_main)
z_inv = np.polyfit(mean_stim[:,1], self_score.inv_adbs_score.values, 1)
f_inv = np.poly1d(z_inv)

ax.scatter(mean_stim[:,0], self_score.main_adbs_score.values, color='tab:red')
ax.scatter(mean_stim[:,1], self_score.inv_adbs_score.values, color='tab:blue')

x_main = np.concatenate([mean_stim[:,0], 
                         [np.min(mean_stim[:,0])-0.02], 
                         [np.max(mean_stim[:,0])+0.02]])
ax.plot(x_main, f_main(x_main), 'tab:red')
x_inv = np.concatenate([mean_stim[:,1], 
                        [np.min(mean_stim[:,1])-0.02], 
                        [np.max(mean_stim[:,1])+0.02]])
ax.plot(x_inv, f_inv(x_inv), 'tab:blue')

ax.set_ylim([3.7, 10.3])
ax.set_ylabel('aDBS rating \n (0-10)  ')
ax.set_xlim([1.6, 2.2])
ax.set_xticks(np.arange(1.6, 2.21, 0.1))
ax.set_xticklabels([1.6,'','','','','',2.2])
ax.set_xlabel('Mean stimulation amplitude \n (sides combined)')
ax.legend(['Movement\n responsive', 'Inverted'], bbox_to_anchor=(0.8, 0.4))

ax.annotate('r = ' + str(np.round(r_inv,2)) 
            + '\np = ' + str(np.round(p_inv,3)), 
            xy=(2.05, 8.7), color='tab:blue')
ax.annotate('r = ' + str(np.round(r_main,2)) 
            + '\np = ' + str(np.round(p_main,3)), 
            xy=(1.62, 4.2), color='tab:red')

plt.tight_layout()

## Minimum dataset provided with manuscript ##
fig6c_mean_stim = pd.DataFrame(mean_stim[:,:2],
                               columns=['movement_responsive', 'inverted'])
# fig6c_mean_stim.to_csv('fig6c_mean_stim.csv', index=False)

## Supplementary Figure 1: Decoder drift effect

In [ ]:
acc = (acc_left + acc_right) / 2

plt.style.use('ggplot')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
fig, ax_left = plt.subplots(1,1, figsize=(5.5,2.8), sharex='row', sharey='row')

ax_right = ax_left.twinx()
ax_left.plot(acc[:,0], color='tab:blue', label='Accuracy')
ax_left.plot(f1, '--', color='tab:blue', label='F1 score')
ax_right.plot(mean_stim[:,0], color='tab:red', label='Mean stimulation')

ax_left.set_xlabel('Session number')
ax_left.set_xlim([-0.5, 11.5])
ax_left.set_xticks(np.arange(12))
ax_left.set_xticklabels([1] + ['']*2 + [4] + ['']*4 + [9] + ['']*2 + [12])
ax_left.set_ylabel('Decoder performance', color='tab:blue')
ax_left.set_ylim([0.3, 0.9])
ax_left.set_yticks(np.arange(0.3,0.91,0.1))
ax_left.set_yticklabels([0.3,'',0.5,'',0.7,'',0.9])
ax_left.legend()

ax_right.set_ylabel('Mean stimulation (mA)', color='tab:red')
ax_right.set_ylim([1.6, 1.9])
ax_right.set_yticks(np.arange(1.6,1.91,0.05))
ax_right.set_yticklabels([1.6,'',1.7,'',1.8,'',1.9])

ax_left.axvline(x=3.5, color='grey', zorder=999)
ax_left.text(3.7, 0.515, 'LH threshold \n update', ha='left')

ax_left.axvline(x=8.5, color='grey', zorder=999)
ax_left.text(8.4, 0.415, 'RH threshold \n update ', ha='right')


plt.tight_layout()

In [ ]:
days_since_training = np.array([282, 286, 288, 306, 384, 385, 
                                386, 387, 420, 429, 434, 435])

plt.style.use('ggplot')
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
fig, ax_left = plt.subplots(1,1, figsize=(5.5,2.8), sharex='row', sharey='row')

ax_left.set_axisbelow(True)
ax_right = ax_left.twinx()
ax_right.set_axisbelow(True)
ax_left.plot(days_since_training, acc[:,0], '-o', color='tab:blue', label='Accuracy')
ax_left.plot(days_since_training, f1, '--o', color='tab:blue', label='F1 score')
ax_right.plot(days_since_training, mean_stim[:,0], '-o', color='tab:red', label='Mean stimulation')

ax_left.set_xlabel('Days since training data collection')
ax_left.set_ylabel('Decoder performance', color='tab:blue')
ax_left.set_ylim([0.3, 0.9])
ax_left.set_yticks(np.arange(0.3,0.91,0.1))
ax_left.set_yticklabels([0.3,'',0.5,'',0.7,'',0.9])
ax_left.legend()

ax_right.set_ylabel('Mean stimulation (mA)', color='tab:red')
ax_right.set_ylim([1.6, 1.9])
ax_right.set_yticks(np.arange(1.6,1.91,0.05))
ax_right.set_yticklabels([1.6,'',1.7,'',1.8,'',1.9])

ax_left.axvline(x=383, color='grey', zorder=-1)
ax_left.text(383, 0.515, 'LH threshold \n update ', ha='right')

ax_left.axvline(x=428, color='grey', zorder=-1)
ax_left.text(428, 0.415, 'RH threshold \n update ', ha='right')

plt.tight_layout()
# plt.savefig('decoder_stability.svg', format='svg')

## Minimum dataset provided with manuscript ##
stability_df = pd.DataFrame(np.vstack([days_since_training, acc[:,0], f1, mean_stim[:,0]]).T, 
                            columns=['day_since_data_collection', 'accuracy', 'f1', 'mean_stim'])
# stability_df.to_csv('S1_stability_df.csv', index=False)

## Behavior

### Figure 6e, Supplementary Figure 3: Impacts on dyskinesia and tremor at rest

In [ ]:
# Read in the data
left_rest_power = pd.read_csv(my_folder + '/rest_data/left_rest_power.csv'
                              ).drop(columns='Unnamed: 0')
right_rest_power = pd.read_csv(my_folder + '/rest_data/right_rest_power.csv'
                               ).drop(columns='Unnamed: 0')

left_rest_psd = pd.read_csv(my_folder + '/rest_data/left_psd.csv'
                            ).drop(columns='Unnamed: 0')
right_rest_psd = pd.read_csv(my_folder + '/rest_data/right_psd.csv'
                             ).drop(columns='Unnamed: 0')

In [ ]:
# statistics: ANCOVA followed by planned contrasts
data = pd.DataFrame({'stim_condition' : np.concatenate(
                                           [left_rest_power['config'].values, 
                                            right_rest_power['config'].values]).astype(str),
                     'dk': np.concatenate([left_rest_power['dk'].values, 
                                           right_rest_power['dk'].values]),
                     'tremor': np.concatenate([left_rest_power['tremor'].values, 
                                               right_rest_power['tremor'].values]),
                     'is_first': [1, 0, 0, 
                                  1, 0, 0, 
                                  0, 0, 1, 
                                  0, 0, 1,
                                  1, 0, 0, 
                                  0, 1, 0, 
                                  0, 1, 0, 
                                  0, 1, 0,
                                  0, 0, 1, 
                                  0, 1, 0, 
                                  1, 0, 0, 
                                  0, 0, 1]*2,
                     'is_second': [0, 1, 0, 
                                   0, 0, 1, 
                                   1, 0, 0, 
                                   0, 1, 0,
                                   0, 0, 1, 
                                   1, 0, 0, 
                                   1, 0, 0, 
                                   0, 0, 1,
                                   0, 1, 0, 
                                   0, 0, 1, 
                                   0, 1, 0, 
                                   1, 0, 0]*2,
                     'session_id': np.concatenate([np.repeat(np.arange(12), 3), 
                                                   np.repeat(np.arange(12), 3)]),
                     'hand' : ['left']*36 + ['right']*36})

for session_id in np.unique(data.session_id):
    mean_dk = np.mean(data[data.session_id==session_id].dk)
    data.loc[data.session_id==session_id, 'dk'] -= mean_dk
    mean_tremor = np.mean(data[data.session_id==session_id].tremor)
    data.loc[data.session_id==session_id, 'tremor'] -= mean_tremor

# Fit the two-way ANCOVA model with interaction term
tremor_model = sm.OLS.from_formula('tremor ~ stim_condition + hand + is_first + is_second + stim_condition:hand', data=data).fit()
dk_model = sm.OLS.from_formula('dk ~ stim_condition + hand + is_first + is_second + stim_condition:hand', data=data).fit()
tremor_anova_table = sm.stats.anova_lm(tremor_model, typ=2)
dk_anova_table = sm.stats.anova_lm(dk_model, typ=2)

# Print the ANOVA table
print('TREMOR: Two-way ANCOVA')
print(tremor_anova_table)
print('\n DYSKINESIA: Two-way ANCOVA')
print(dk_anova_table)


## Minimum dataset provided with manuscript ##
# fig6d_rep_rates = data.to_csv('fig6e_S3_dk_tremor.csv', index=False)

In [ ]:
tremor_model = sm.OLS.from_formula('tremor ~ stim_condition + is_first', data=data).fit()
dk_model = sm.OLS.from_formula('dk ~ stim_condition + hand + is_first', data=data).fit()

# Print the ANCOVA coefficients after including only significant terms
print('TREMOR: ANCOVA significant coefficients')
print(tremor_model.params)
print('\n DYSKINESIA: ANCOVA significant coefficients')
print(dk_model.params)

In [ ]:
# control for the `is_first` covariate
data['tremor_adjusted'] = data.tremor \
                          - tremor_model.params['is_first']*data.is_first
data['dk_adjusted'] = data.dk \
                      - dk_model.params['hand[T.right]']*(data.hand=='right') \
                      - dk_model.params['is_first']*data.is_first

ttest_tremor_1_2 = ttest_ind(data.tremor_adjusted[(data.stim_condition.values=='1')], 
                             data.tremor_adjusted[(data.stim_condition.values=='2')])
ttest_tremor_1_3 = ttest_ind(data.tremor_adjusted[(data.stim_condition.values=='1')], 
                             data.tremor_adjusted[(data.stim_condition.values=='3')])
ttest_tremor_2_3 = ttest_ind(data.tremor_adjusted[(data.stim_condition.values=='2')], 
                             data.tremor_adjusted[(data.stim_condition.values=='3')])
ttest_dk_1_2 = ttest_ind(data.dk_adjusted[(data.stim_condition.values=='1')], 
                         data.dk_adjusted[(data.stim_condition.values=='2')])
ttest_dk_1_3 = ttest_ind(data.dk_adjusted[(data.stim_condition.values=='1')], 
                         data.dk_adjusted[(data.stim_condition.values=='3')])
ttest_dk_2_3 = ttest_ind(data.dk_adjusted[(data.stim_condition.values=='2')], 
                         data.dk_adjusted[(data.stim_condition.values=='3')])

print('PLANNED CONTRASTS \n Covariate-adjusted, exclude non-sig interaction')

print('\n TREMOR')
print('\t Stim condition 1 vs 2: ', ttest_tremor_1_2)
print('\t Stim condition 1 vs 3: ', ttest_tremor_1_3)
print('\t Stim condition 2 vs 3: ', ttest_tremor_2_3)

print('\n DYSKINESIA')
print('\t Stim condition 1 vs 2: ', ttest_dk_1_2)
print('\t Stim condition 1 vs 3: ', ttest_dk_1_3)
print('\t Stim condition 2 vs 3: ', ttest_dk_2_3)

In [ ]:
# xxx_rest_power dataframe should contain columns: 
# config, dk, tremor
# REORGANIZE TO PUT BOTH HANDS TOGETHER AND MAKE THE SUBPLOTS DK AND TREMOR INSTEAD

fig, ax = plt.subplots(1,2, figsize=(5.5,2.3), sharex=True, sharey=True)

left_order = np.zeros(19, int)
left_order[[0,7,14]] = [3,1,2]
right_order = np.zeros(19, int)
right_order[[3,10,17]] = [3,1,2]

sns.swarmplot(data=left_rest_power, 
              x='config', y='dk', 
              order=left_order, alpha=0.8, size=4, hue='config', ax=ax[0])
sns.boxplot(data=left_rest_power, 
            x='config', y='dk', 
            order=np.roll(left_order, 1), dodge=False, hue='config', width=1, whis=10, ax=ax[0])

sns.swarmplot(data=right_rest_power, 
              x='config', y='dk', 
              order=np.roll(right_order, 1), alpha=0.8, size=4, hue='config', ax=ax[0])
sns.boxplot(data=right_rest_power, 
            x='config', y='dk', 
            order=right_order, dodge=False, hue='config', width=1, whis=10, ax=ax[0])

sns.swarmplot(data=left_rest_power, 
              x='config', y='tremor', 
              order=left_order, alpha=0.8, size=4, hue='config', ax=ax[1])
sns.boxplot(data=left_rest_power, 
            x='config', y='tremor', 
            order=np.roll(left_order, 1), dodge=False, hue='config', width=1, whis=10, ax=ax[1])

sns.swarmplot(data=right_rest_power, 
              x='config', y='tremor', 
              order=np.roll(right_order, 1), alpha=0.8, size=4, hue='config', ax=ax[1])
sns.boxplot(data=right_rest_power, 
            x='config', y='tremor', 
            order=right_order, dodge=False, hue='config', width=1, whis=10, ax=ax[1])

# styling
ax[0].set_title('Dyskinesia (1-4Hz)')
ax[1].set_title('Tremor (4-7Hz)')
ax[0].set_xticks([2, 9, 16])
ax[0].set_xticklabels(['Constant', 'Movement \n responsive ', 'Inverted'])
ax[0].set_xlabel('')
ax[1].set_xlabel('')
ax[0].set_xlim([-1, 20])

ax[0].set_ylabel('Band power \n log$_{10}(G^2/Hz)$')
ax[1].set_ylabel('')
ax[0].set_yticks(np.arange(-7,-0.9,1))
ax[0].set_yticklabels([-7, '', '', '', '', '', -1])
ax[0].set_ylim([-7, -1])

for i in range(2):
    ax[i].get_legend().remove()
    
# annotate to show significance - DK
max_dk_constant = np.max([np.max(left_rest_power.dk[(left_rest_power.config.values==3)]),
                          np.max(right_rest_power.dk[(right_rest_power.config.values==3)])])
max_dk_responsive = np.max([np.max(left_rest_power.dk[(left_rest_power.config.values==1)]),
                            np.max(right_rest_power.dk[(right_rest_power.config.values==1)])])
max_dk_inverted = np.max([np.max(left_rest_power.dk[(left_rest_power.config.values==2)]),
                          np.max(right_rest_power.dk[(right_rest_power.config.values==2)])])
ax[0].axhline(y=max_dk_constant+0.3, xmin=2/21, xmax=4/21, color='k', linewidth=1)
ax[0].axhline(y=max_dk_responsive+0.3, xmin=9/21, xmax=11/21, color='k', linewidth=1)
ax[0].axhline(y=np.max([max_dk_constant, max_dk_responsive])+0.6, 
              xmin=3/21, xmax=10/21, color='k', linewidth=1)
ax[0].axvline(x=2, 
              ymin=(7+max_dk_constant+0.3)/6, 
              ymax=(7+max_dk_constant+0.6)/6, 
              color='k', linewidth=1)
ax[0].axvline(x=9, 
              ymin=(7+max_dk_responsive+0.3)/6, 
              ymax=(7+max_dk_inverted+0.6)/6, 
              color='k', linewidth=1)
ax[0].annotate('*', xy=[5.2, max_dk_constant+0.6])

ax[0].axhline(y=max_dk_inverted+0.3, xmin=16/21, xmax=18/21, color='k', linewidth=1)
ax[0].axhline(y=np.max([max_dk_inverted, max_dk_responsive])+0.6, 
              xmin=10/21, xmax=17/21, color='k', linewidth=1)
ax[0].axvline(x=16, 
              ymin=(7+max_dk_inverted+0.3)/6, 
              ymax=(7+max_dk_inverted+0.6)/6, 
              color='k', linewidth=1)
ax[0].annotate('*', xy=[12.2, max_dk_inverted+0.6])

# annotate to show significance - tremor
max_tremor_constant = np.max([np.max(left_rest_power.tremor[(left_rest_power.config.values==3)]),
                              np.max(right_rest_power.tremor[(right_rest_power.config.values==3)])])
max_tremor_responsive = np.max([np.max(left_rest_power.tremor[(left_rest_power.config.values==1)]),
                                np.max(right_rest_power.tremor[(right_rest_power.config.values==1)])])
max_tremor_inverted = np.max([np.max(left_rest_power.tremor[(left_rest_power.config.values==2)]),
                              np.max(right_rest_power.tremor[(right_rest_power.config.values==2)])])
ax[1].axhline(y=max_tremor_constant+0.3, xmin=2/21, xmax=4/21, color='k', linewidth=1)
ax[1].axhline(y=max_tremor_responsive+0.3, xmin=9/21, xmax=11/21, color='k', linewidth=1)
ax[1].axhline(y=np.max([max_tremor_constant, max_tremor_responsive])+0.6, 
              xmin=3/21, xmax=10/21, color='k', linewidth=1)
ax[1].axvline(x=2, 
              ymin=(7+max_tremor_constant+0.3)/6, 
              ymax=(7+max_tremor_constant+0.6)/6, 
              color='k', linewidth=1)
ax[1].axvline(x=9, 
              ymin=(7+max_tremor_responsive+0.3)/6, 
              ymax=(7+max_tremor_inverted+0.6)/6, 
              color='k', linewidth=1)
ax[1].annotate('*', xy=[5.2, max_tremor_constant+0.6])

ax[1].axhline(y=max_tremor_inverted+0.3, xmin=16/21, xmax=18/21, color='k', linewidth=1)
ax[1].axhline(y=np.max([max_tremor_inverted, max_tremor_responsive])+0.6, 
              xmin=10/21, xmax=17/21, color='k', linewidth=1)
ax[1].axvline(x=16, 
              ymin=(7+max_tremor_inverted+0.3)/6, 
              ymax=(7+max_tremor_inverted+0.6)/6, 
              color='k', linewidth=1)
ax[1].annotate('*', xy=[12.2, max_tremor_inverted+0.6])

plt.tight_layout()

### Figure 6d: Impacts on cyclic movement speeds

In [ ]:
# Read in the data
# finger_tap_data = pd.read_csv(my_folder + '/finger_tapping_data.csv')
nose_tap_data = pd.read_pickle(my_folder + '/nose_tapping_data_pickle.csv')
wrist_rot_data = pd.read_pickle(my_folder + '/wrist_rotation_data_pickle.csv')

In [ ]:
# statistics: ANCOVA followed by planned contrasts
session_id = np.repeat(np.arange(12), 6)

stim_condition = np.ones(72).astype(int)
stim_condition[wrist_rot_data.stim_config.values == 'inverted'] = 2
stim_condition[wrist_rot_data.stim_config.values == 'open_loop'] = 3

hand = wrist_rot_data.task_side.values

is_first = [1, 0, 0, 1, 0, 0,
            1, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 1,
            0, 0, 1, 0, 0, 1,
            1, 0, 0, 1, 0, 0,
            0, 1, 0, 0, 1, 0,
            0, 1, 0, 0, 1, 0,
            0, 1, 0, 0, 1, 0,
            0, 0, 1, 0, 0, 1,
            0, 1, 0, 0, 1, 0,
            1, 0, 0, 1, 0, 0,
            0, 0, 1, 0, 0, 1]

is_second = [0, 1, 0, 0, 1, 0,
             0, 0, 1, 0, 0, 1,
             1, 0, 0, 1, 0, 0,
             0, 1, 0, 0, 1, 0,
             0, 0, 1, 0, 0, 1,
             1, 0, 0, 1, 0, 0, 
             1, 0, 0, 1, 0, 0, 
             0, 0, 1, 0, 0, 1,
             0, 1, 0, 0, 1, 0,
             0, 0, 1, 0, 0, 1,
             0, 1, 0, 0, 1, 0,
             1, 0, 0, 1, 0, 0]

nose_tap_rate = [1/np.mean(nose_tap_data.peak_to_peak_time[i]) 
                 for i in range(14)] \
                + [1/np.mean(nose_tap_data.peak_to_peak_time[i]) 
                   for i in np.arange(13,71)]

wrist_rot_rate = [1/np.mean(wrist_rot_data.peak_to_peak_time[i]) 
                  for i in range(72)]

nose_tap_size = [np.mean(nose_tap_data.peak_to_trough_distance[i]) 
                 for i in range(14)] \
                + [np.mean(nose_tap_data.peak_to_trough_distance[i]) 
                   for i in np.arange(13,71)]

wrist_rot_size = [np.mean(wrist_rot_data.peak_to_trough_distance[i]) 
                  for i in range(72)]

data = pd.DataFrame({'session_id': session_id,
                     'stim_condition' : stim_condition.astype(str),
                     'hand' : hand,
                     'is_first': is_first,
                     'is_second': is_second,
                     'nose_tap_rate': nose_tap_rate,
                     'nose_tap_rate_session_adj': nose_tap_rate,
                     'wrist_rot_rate': wrist_rot_rate,
                     'wrist_rot_rate_session_adj': wrist_rot_rate,})

# data = data.drop([14]).reset_index()

for session_id in np.unique(data.session_id):
    mean_nose_tap_rate = np.mean(data[data.session_id==session_id].nose_tap_rate)
    data.loc[data.session_id==session_id, 'nose_tap_rate_session_adj'] -= mean_nose_tap_rate
    
    mean_wrist_rot_rate = np.mean(data[data.session_id==session_id].wrist_rot_rate)
    data.loc[data.session_id==session_id, 'wrist_rot_rate_session_adj'] -= mean_wrist_rot_rate

# Fit the two-way ANCOVA model with interaction term
nose_tap_rate_model = sm.OLS.from_formula('nose_tap_rate_session_adj ~ stim_condition + hand + is_first + is_second + stim_condition:hand', data=data).fit()
wrist_rot_rate_model = sm.OLS.from_formula('wrist_rot_rate_session_adj ~ stim_condition + hand + is_first + is_second + stim_condition:hand', data=data).fit()

nose_tap_rate_anova_table = sm.stats.anova_lm(nose_tap_rate_model, typ=2)
wrist_rot_rate_anova_table = sm.stats.anova_lm(wrist_rot_rate_model, typ=2)

# Print the ANOVA table
print('NOSE TAP RATE: Two-way ANCOVA')
print(nose_tap_rate_anova_table)
print('\n WRIST ROTATION RATE: Two-way ANCOVA')
print(wrist_rot_rate_anova_table)

In [ ]:
nose_tap_rate_model = sm.OLS.from_formula('nose_tap_rate_session_adj ~ stim_condition + stim_condition:hand', data=data).fit()
wrist_rot_rate_model = sm.OLS.from_formula('wrist_rot_rate_session_adj ~ stim_condition + hand + is_second + stim_condition:hand', data=data).fit()

# Print the ANCOVA coefficients after including only significant terms
print('NOSE TAP RATE: ANCOVA significant coefficients')
print(nose_tap_rate_model.params)
print('\n WRIST ROTATION RATE: ANCOVA significant coefficients')
print(wrist_rot_rate_model.params)

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(5.1,2.3), sharex=True, sharey=False)
data['stim_condition'] = pd.to_numeric(data['stim_condition'])

left_order = np.zeros(19, int)
left_order[[0,7,14]] = [3,1,2]
right_order = np.zeros(19, int)
right_order[[3,10,17]] = [3,1,2]

# wrist rotations
sns.swarmplot(data=data[data.hand=='left'], 
              x='stim_condition', y='wrist_rot_rate', 
              order=left_order, alpha=0.8, size=4, hue='stim_condition', ax=ax[0])
sns.boxplot(data=data[data.hand=='left'], 
            x='stim_condition', y='wrist_rot_rate', 
            order=np.roll(left_order, 1), dodge=False, hue='stim_condition', width=1, whis=10, ax=ax[0])

sns.swarmplot(data=data[data.hand=='right'], 
              x='stim_condition', y='wrist_rot_rate', 
              order=np.roll(right_order, 1), alpha=0.8, size=4, hue='stim_condition', ax=ax[0])
sns.boxplot(data=data[data.hand=='right'], 
            x='stim_condition', y='wrist_rot_rate', 
            order=right_order, dodge=False, hue='stim_condition', width=1, whis=10, ax=ax[0])

# nose tapping
sns.swarmplot(data=data[data.hand=='left'], 
              x='stim_condition', y='nose_tap_rate', 
              order=left_order, alpha=0.8, size=4, hue='stim_condition', ax=ax[1])
sns.boxplot(data=data[data.hand=='left'], 
            x='stim_condition', y='nose_tap_rate', 
            order=np.roll(left_order, 1), dodge=False, hue='stim_condition', width=1, whis=10, ax=ax[1])

sns.swarmplot(data=data[data.hand=='right'], 
              x='stim_condition', y='nose_tap_rate', 
              order=np.roll(right_order, 1), alpha=0.8, size=4, hue='stim_condition', ax=ax[1])
sns.boxplot(data=data[data.hand=='right'], 
            x='stim_condition', y='nose_tap_rate', 
            order=right_order, dodge=False, hue='stim_condition', width=1, whis=10, ax=ax[1])

# styling
ax[0].set_title('Finger tapping')
ax[0].set_title('Wrist rotations')
ax[1].set_title('Nose tapping')
ax[0].set_xticks([2, 9, 16])
ax[0].set_xticklabels(['Constant', 'Movement \n responsive ', 'Inverted'])
ax[0].set_xlim([-1, 19])

ax[0].set_ylabel('Repetition rate (Hz)')
ax[1].set_ylabel('')

ax[0].set_ylim([1.8, 4.0])
ax[1].set_ylim([0.6, 3.4])

for i in range(2):
    ax[i].get_legend().remove()
    ax[i].set_xlabel('')
    
# annotate to show significance - wrist rotation rate
max_WRR_constant_left = np.max(data.wrist_rot_rate[(data.stim_condition.values==3) & (data.hand.values=='left')])
max_WRR_constant_right = np.max(data.wrist_rot_rate[(data.stim_condition.values==3) & (data.hand.values=='right')])
max_WRR_responsive_left = np.max(data.wrist_rot_rate[(data.stim_condition.values==1) & (data.hand.values=='left')])
max_WRR_responsive_right = np.max(data.wrist_rot_rate[(data.stim_condition.values==1) & (data.hand.values=='right')])
max_WRR_inverted_left = np.max(data.wrist_rot_rate[(data.stim_condition.values==2) & (data.hand.values=='left')])
max_WRR_inverted_right = np.max(data.wrist_rot_rate[(data.stim_condition.values==2) & (data.hand.values=='right')])

ax[0].axhline(y=max_WRR_responsive_right+0.2, xmin=11/20, xmax=18/20, color='k', linewidth=1)
ax[0].axhline(y=max_WRR_constant_right+0.3, xmin=4/20, xmax=18/20, color='k', linewidth=1)

ax[0].axvline(x=3, 
              ymin=(-1.8+max_WRR_constant_right+0.15)/2.2, 
              ymax=(-1.8+max_WRR_constant_right+0.3)/2.2, 
              color='k', linewidth=1)
ax[0].axvline(x=10, 
              ymin=(-1.8+max_WRR_responsive_right+0.15)/2.2, 
              ymax=(-1.8+max_WRR_responsive_right+0.2)/2.2, 
              color='k', linewidth=1)
ax[0].axvline(x=17, 
              ymin=(-1.8+max_WRR_inverted_right+0.15)/2.2, 
              ymax=(-1.8+max_WRR_constant_right+0.3)/2.2, 
              color='k', linewidth=1)

ax[0].annotate('*', xy=[13.2, max_WRR_responsive_right+0.155])
ax[0].annotate('*', xy=[9.2, max_WRR_constant_right+0.255])

# annotate to show significance - nose tap rate
max_NTR_constant_left = np.max(data.nose_tap_rate[(data.stim_condition.values==3) & (data.hand.values=='left')])
max_NTR_constant_right = np.max(data.nose_tap_rate[(data.stim_condition.values==3) & (data.hand.values=='right')])
max_NTR_responsive_left = np.max(data.nose_tap_rate[(data.stim_condition.values==1) & (data.hand.values=='left')])
max_NTR_responsive_right = np.max(data.nose_tap_rate[(data.stim_condition.values==1) & (data.hand.values=='right')])
max_NTR_inverted_left = np.max(data.nose_tap_rate[(data.stim_condition.values==2) & (data.hand.values=='left')])
max_NTR_inverted_right = np.max(data.nose_tap_rate[(data.stim_condition.values==2) & (data.hand.values=='right')])

ax[1].axhline(y=max_NTR_responsive_right+0.3, xmin=11/20, xmax=18/20, color='k', linewidth=1)
ax[1].axhline(y=max_NTR_responsive_right+0.6, xmin=4/20, xmax=18/20, color='k', linewidth=1)

ax[1].axvline(x=3, 
              ymin=(-0.6+max_NTR_constant_right+0.2)/2.8, 
              ymax=(-0.6+max_NTR_responsive_right+0.6)/2.8, 
              color='k', linewidth=1)
ax[1].axvline(x=10, 
              ymin=(-0.6+max_NTR_responsive_right+0.2)/2.8, 
              ymax=(-0.6+max_NTR_responsive_right+0.3)/2.8, 
              color='k', linewidth=1)
ax[1].axvline(x=17, 
              ymin=(-0.6+max_NTR_inverted_right+0.2)/2.8, 
              ymax=(-0.6+max_NTR_responsive_right+0.6)/2.8, 
              color='k', linewidth=1)

ax[1].annotate('*', xy=[9.2, max_NTR_responsive_right+0.601])
ax[1].annotate('*', xy=[13.2, max_NTR_responsive_right+0.301])


plt.tight_layout()
# fig6d_rep_rates = data.to_csv('fig6d_rep_rates.csv', index=False)

In [ ]:
# control for the `is_first` covariate
data['wrist_rot_rate_cov_adj'] = data.wrist_rot_rate_session_adj \
                                 - wrist_rot_rate_model.params['hand[T.right]']*(data.hand=='right') \
                                 - wrist_rot_rate_model.params['is_second']*data.is_second
data['nose_tap_rate_cov_adj'] = data.nose_tap_rate_session_adj                             

comp_list = [[1, 'left',  2, 'left'],
             [1, 'left',  3, 'left'],
             [2, 'left',  3, 'left'],
             [1, 'right', 2, 'right'],
             [1, 'right', 3, 'right'],
             [2, 'right', 3, 'right']]

NTR_tests = []
WRR_tests = []
for i in range(6):
    NTR_tests.append(ttest_ind(data.nose_tap_rate_session_adj[(data.stim_condition.values==comp_list[i][0]) 
                                                              & (data.hand.values==comp_list[i][1])], 
                               data.nose_tap_rate_session_adj[(data.stim_condition.values==comp_list[i][2])
                                                              & (data.hand.values==comp_list[i][3])]))
    WRR_tests.append(ttest_ind(data.wrist_rot_rate_cov_adj[(data.stim_condition.values==comp_list[i][0]) 
                                                           & (data.hand.values==comp_list[i][1])], 
                               data.wrist_rot_rate_cov_adj[(data.stim_condition.values==comp_list[i][2])
                                                           & (data.hand.values==comp_list[i][3])]))

print('PLANNED CONTRASTS \n Covariate-adjusted, exclude non-sig interaction')

print('\n\n LEFT HAND')
print('\n WRIST ROTATION RATE')
print('\t Stim condition 1 vs 2: ', WRR_tests[0])
print('\t Stim condition 1 vs 3: ', WRR_tests[1])
print('\t Stim condition 2 vs 3: ', WRR_tests[2])

print('\n FINGER-TO-NOSE RATE')
print('\t Stim condition 1 vs 2: ', NTR_tests[0])
print('\t Stim condition 1 vs 3: ', NTR_tests[1])
print('\t Stim condition 2 vs 3: ', NTR_tests[2])


print('\n\n RIGHT HAND')
print('\n WRIST ROTATION RATE')
print('\t Stim condition 1 vs 2: ', WRR_tests[3])
print('\t Stim condition 1 vs 3: ', WRR_tests[4])
print('\t Stim condition 2 vs 3: ', WRR_tests[5])

print('\n FINGER-TO-NOSE RATE')
print('\t Stim condition 1 vs 2: ', NTR_tests[3])
print('\t Stim condition 1 vs 3: ', NTR_tests[4])
print('\t Stim condition 2 vs 3: ', NTR_tests[5])

### Figure 7: Impacts on typing performance

In [ ]:
# Read in the data
keypress_df = pd.read_csv(my_folder + '/keypress_data.csv')
keypress_df = keypress_df.astype({'config': str})
keypress_df['session_id'] = np.repeat(np.arange(12), 3).astype(str)
keypress_df['is_first'] = (keypress_df['epoch'].values == 0).astype(int)
keypress_df['is_second'] = (keypress_df['epoch'].values == 1).astype(int)
keypress_df['kp_duration_session_adjusted'] = keypress_df['mean_duration']
keypress_df['speed_session_adjusted'] = keypress_df['mean_typing_speed']
keypress_df['bs_rate_session_adjusted'] = keypress_df['backspace_per_press']
for date in np.unique(keypress_df.date):
    mean_duration = np.mean(keypress_df[keypress_df.date==date].mean_duration)
    keypress_df.loc[keypress_df.date==date, 'kp_duration_session_adjusted'] -= mean_duration
    mean_speed = np.mean(keypress_df[keypress_df.date==date].mean_typing_speed)
    keypress_df.loc[keypress_df.date==date, 'speed_session_adjusted'] -= mean_speed
    mean_BS = np.mean(keypress_df[keypress_df.date==date].backspace_per_press)
    keypress_df.loc[keypress_df.date==date, 'bs_rate_session_adjusted'] -= mean_BS

In [ ]:
# Fit the two-way ANCOVA model with interaction term
press_duration_mdl = sm.OLS.from_formula('kp_duration_session_adjusted ~ config + is_first + is_second', data=keypress_df).fit()
typing_speed_mdl = sm.OLS.from_formula('speed_session_adjusted ~ config + is_first + is_second', data=keypress_df).fit()
backspace_mdl = sm.OLS.from_formula('bs_rate_session_adjusted ~ config + is_first + is_second', data=keypress_df).fit()
press_duration_anova_table = sm.stats.anova_lm(press_duration_mdl, typ=2)
typing_speed_anova_table = sm.stats.anova_lm(typing_speed_mdl, typ=2)
backspace_anova_table = sm.stats.anova_lm(backspace_mdl, typ=2)

# Print the ANOVA table
print('KEYPRESS DURATION: One-way ANCOVA')
print(press_duration_anova_table)
print('\n TYPING SPEED: One-way ANCOVA')
print(typing_speed_anova_table)
print('\n BACKSPACE RATE: One-way ANCOVA')
print(backspace_anova_table)

## Minimum dataset provided with manuscript ##
keypress_df = keypress_df.drop('date', axis=1)
# keypress_df.to_csv('fig7_typing.csv', index=False)

In [ ]:
press_duration_mdl = sm.OLS.from_formula('kp_duration_session_adjusted ~ config + is_first + is_second', data=keypress_df).fit()
typing_speed_mdl = sm.OLS.from_formula('speed_session_adjusted ~ config + is_first + is_second', data=keypress_df).fit()
backspace_mdl = sm.OLS.from_formula('bs_rate_session_adjusted ~ config', data=keypress_df).fit()

# Print the ANCOVA coefficients after including only significant terms
print('KEYPRESS DURATION: ANCOVA significant coefficients')
print(press_duration_mdl.params)
print('\n TYPING SPEED: ANCOVA significant coefficients')
print(typing_speed_mdl.params)
print('\n BACKSPACE RATE: ANCOVA significant coefficients')
print(backspace_mdl.params)

In [ ]:
# control for the significant covariate effects
keypress_df['kp_duration_cov_adjusted'] = keypress_df.kp_duration_session_adjusted \
                                          - press_duration_mdl.params['is_first']*keypress_df.is_first \
                                          - press_duration_mdl.params['is_second']*keypress_df.is_second
keypress_df['speed_cov_adjusted'] = keypress_df.speed_session_adjusted \
                                    - typing_speed_mdl.params['is_first']*keypress_df.is_first \
                                    - typing_speed_mdl.params['is_second']*keypress_df.is_second

ttest_press_duration_1_2 = ttest_ind(keypress_df.kp_duration_cov_adjusted[(keypress_df.config.values=='1')], 
                                     keypress_df.kp_duration_cov_adjusted[(keypress_df.config.values=='2')])
ttest_press_duration_1_3 = ttest_ind(keypress_df.kp_duration_cov_adjusted[(keypress_df.config.values=='1')], 
                                     keypress_df.kp_duration_cov_adjusted[(keypress_df.config.values=='3')])
ttest_press_duration_2_3 = ttest_ind(keypress_df.kp_duration_cov_adjusted[(keypress_df.config.values=='2')], 
                                     keypress_df.kp_duration_cov_adjusted[(keypress_df.config.values=='3')])
ttest_typing_speed_1_2 = ttest_ind(keypress_df.speed_cov_adjusted[(keypress_df.config.values=='1')], 
                                   keypress_df.speed_cov_adjusted[(keypress_df.config.values=='2')])
ttest_typing_speed_1_3 = ttest_ind(keypress_df.speed_cov_adjusted[(keypress_df.config.values=='1')], 
                                   keypress_df.speed_cov_adjusted[(keypress_df.config.values=='3')])
ttest_typing_speed_2_3 = ttest_ind(keypress_df.speed_cov_adjusted[(keypress_df.config.values=='2')], 
                                   keypress_df.speed_cov_adjusted[(keypress_df.config.values=='3')])
ttest_backspace_duration_1_2 = ttest_ind(keypress_df.backspace_per_press[(keypress_df.config.values=='1')], 
                                         keypress_df.backspace_per_press[(keypress_df.config.values=='2')])
ttest_backspace_duration_1_3 = ttest_ind(keypress_df.backspace_per_press[(keypress_df.config.values=='1')], 
                                         keypress_df.backspace_per_press[(keypress_df.config.values=='3')])
ttest_backspace_duration_2_3 = ttest_ind(keypress_df.backspace_per_press[(keypress_df.config.values=='2')], 
                                         keypress_df.backspace_per_press[(keypress_df.config.values=='3')])

print('PLANNED CONTRASTS \n Covariate-adjusted, exclude non-sig interaction')

print('\n KEYPRESS DURATION')
print('\t Stim condition 1 vs 2: ', ttest_press_duration_1_2)
print('\t Stim condition 1 vs 3: ', ttest_press_duration_1_3)
print('\t Stim condition 2 vs 3: ', ttest_press_duration_2_3)

print('\n TYPING SPEED')
print('\t Stim condition 1 vs 2: ', ttest_typing_speed_1_2)
print('\t Stim condition 1 vs 3: ', ttest_typing_speed_1_3)
print('\t Stim condition 2 vs 3: ', ttest_typing_speed_2_3)

print('\n BACKSPACE RATE')
print('\t Stim condition 1 vs 2: ', ttest_backspace_duration_1_2)
print('\t Stim condition 1 vs 3: ', ttest_backspace_duration_1_3)
print('\t Stim condition 2 vs 3: ', ttest_backspace_duration_2_3)

In [ ]:
# keypress_df dataframe should contain columns: 
# mean_typing_speed, mean_duration, backspace_per_press

keypress_df = keypress_df.astype({'config': int})
fig, ax = plt.subplots(3,1, figsize=(3.3,5.3), sharex=True, sharey=False)

order = np.zeros(10, int)
order[[0,4,8]] = [3,1,2]

sns.swarmplot(data=keypress_df, 
              x='config', y='mean_duration', 
              order=np.roll(order, 1), alpha=0.8, size=4, hue='config', ax=ax[0])
sns.boxplot(data=keypress_df, 
            x='config', y='mean_duration', 
            order=order, dodge=False, hue='config', width=1, whis=10, ax=ax[0])

sns.swarmplot(data=keypress_df, 
              x='config', y='mean_typing_speed', 
              order=np.roll(order, 1), alpha=0.8, size=4, hue='config', ax=ax[1])
sns.boxplot(data=keypress_df, 
            x='config', y='mean_typing_speed', 
            order=order, dodge=False, hue='config', width=1, whis=10, ax=ax[1])

sns.swarmplot(data=keypress_df, 
              x='config', y='backspace_per_press', 
              order=np.roll(order, 1), alpha=0.8, size=4, hue='config', ax=ax[2])
sns.boxplot(data=keypress_df, 
            x='config', y='backspace_per_press', 
            order=order, dodge=False, hue='config', width=1, whis=10, ax=ax[2])

# styling
ax[0].set_ylabel('Mean keypress \n duration (s)')
ax[1].set_ylabel('Typing speed \n (KP/s)')
ax[2].set_ylabel('Backspace rate \n (BS/KP)')
ax[2].set_xticks([0.4, 4.4, 8.4])
ax[2].set_xticklabels(['Constant', 'Movement \n responsive ', 'Inverted'])
ax[0].set_xlabel('')
ax[1].set_xlabel('')
ax[2].set_xlabel('')
ax[0].set_xlim([-1.5, 10])

ax[0].set_ylim([0.115, 0.185])
ax[0].set_yticks([0.12, 0.14, 0.16, 0.18])
ax[0].set_yticklabels(['0.12', '', '', '0.18'])
ax[1].set_ylim([2.75, 6.25])
ax[1].set_yticks([3,4,5,6])
ax[1].set_yticklabels(['3', '', '', '6'])
ax[2].set_ylim([0.05, 0.175])
ax[2].set_yticks(np.arange(0.05,0.176,0.025))
ax[2].set_yticklabels(['0.050', '', '', '', '', '0.175'])

for i in range(3):
    ax[i].get_legend().remove()
    
# annotate to show significance - keypress duration
max_duration_constant = np.max(keypress_df.mean_duration[(keypress_df.config.values==3)])
max_duration_responsive = np.max(keypress_df.mean_duration[(keypress_df.config.values==1)])
max_duration_inverted = np.max(keypress_df.mean_duration[(keypress_df.config.values==2)])

ax[0].axhline(y=max_duration_constant+0.008, xmin=1.9/11.5, xmax=5.9/11.5, color='k', linewidth=1)
ax[0].axhline(y=max_duration_inverted+0.008, xmin=5.9/11.5, xmax=9.9/11.5, color='k', linewidth=1)
ax[0].axvline(x=0.4, 
              ymin=(max_duration_constant+0.004-0.115)/0.07, 
              ymax=(max_duration_constant+0.008-0.115)/0.07, 
              color='k', linewidth=1)
ax[0].axvline(x=4.4, 
              ymin=(max_duration_responsive+0.004-0.115)/0.07, 
              ymax=(max_duration_inverted+0.008-0.115)/0.07, 
              color='k', linewidth=1)
ax[0].axvline(x=8.4, 
              ymin=(max_duration_inverted+0.004-0.115)/0.07, 
              ymax=(max_duration_inverted+0.008-0.115)/0.07, 
              color='k', linewidth=1)
ax[0].annotate('*', xy=[2.4, max_duration_constant+0.009])
ax[0].annotate('*', xy=[6.4, max_duration_inverted+0.009])

# annotate to show significance - typing speed
max_speed_responsive = np.max(keypress_df.mean_typing_speed[(keypress_df.config.values==1)])
max_speed_inverted = np.max(keypress_df.mean_typing_speed[(keypress_df.config.values==2)])

ax[1].axhline(y=max_speed_responsive+0.4, xmin=5.9/11.5, xmax=9.9/11.5, color='k', linewidth=1)
ax[1].axvline(x=4.4, 
              ymin=(max_speed_responsive+0.2-2.75)/3.5, 
              ymax=(max_speed_responsive+0.4-2.75)/3.5, 
              color='k', linewidth=1)
ax[1].axvline(x=8.4, 
              ymin=(max_speed_inverted+0.2-2.75)/3.5, 
              ymax=(max_speed_responsive+0.4-2.75)/3.5, 
              color='k', linewidth=1)
ax[1].annotate('*', xy=[6.4, max_speed_responsive+0.45])

plt.tight_layout()